In [1]:
from datetime import date, timedelta 
import requests
#download and write the csv of Ontario COVID-19 data
req = requests.get("https://data.ontario.ca/dataset/1115d5fe-dd84-4c69-b5ed-05bf0c0a0ff9/resource/d1bfe1ad-6575-4352-8302-09ca81f7ddfc/download/cases_by_status_and_phu.csv")
url_content = req.content
csv_file = open('cases_by_status_and_phu.csv', 'wb')

csv_file.write(url_content)
csv_file.close()

#get today's and yesterday's date info
today = date.today()
dateToday = today.strftime("%Y""-""%m""-""%d")
yesterday = date.today() - timedelta(1)
dateYes = yesterday.strftime("%Y""-""%m""-""%d")

print(dateToday)
print(dateYes)

2021-04-10
2021-04-09


In [2]:
import pandas as pd
#load into a pandas dataframe and set datetime format to query
df = pd.read_csv("https://data.ontario.ca/dataset/1115d5fe-dd84-4c69-b5ed-05bf0c0a0ff9/resource/d1bfe1ad-6575-4352-8302-09ca81f7ddfc/download/cases_by_status_and_phu.csv", header=0)
df['FILE_DATE'] = pd.to_datetime(df['FILE_DATE'])

In [3]:
# Filter for today's and yesterday's dates
phuCasesToday = df[(df['FILE_DATE'] == dateToday)]
phuCasesYes = df[(df['FILE_DATE'] == dateYes)]

In [4]:
# Convert possible dataframes to csv and read csvs for the most recent set available
if phuCasesToday.empty:
    phuCasesYes.to_csv('yesCases.csv')
    dfcurrent = pd.read_csv('yesCases.csv')
else:
    phuCasesToday.to_csv('todayCases.csv')
    dfcurrent = pd.read_csv('todayCases.csv')

In [5]:
# Connect to GIS
from arcgis.gis import GIS
gis = GIS("Home")

# Delete existing layer when the layers already exist from previous running
feature_search = gis.content.search(query = 'Current COVID-19 Cases by Ontario PHU', item_type = "Feature Layer Collection")
if len(feature_search) > 0:
    for item in feature_search:
        item.delete()
        print("Deleted existing " + str(item) + " Feature Layer Collection")

Deleted existing <Item title:"Current COVID-19 Cases by Ontario PHU" type:Feature Layer Collection owner:razewski.n_CPS> Feature Layer Collection


In [6]:
#get shapefiles for Ontario Public Health Regions
ontarioPHUs = gis.content.get("c2fa5249b0c2404ea8132c051d934224")
ontarioPHUsLayer = ontarioPHUs.layers[0]

In [7]:
# Import Ontario PHU Boundaries into a spatially enabled dataframe
sdf = pd.DataFrame.spatial.from_layer(ontarioPHUsLayer)

In [8]:
# Merge the Ontario PHU SEDF with the cases dataframe by PHU identification number 
currentCasesByPHU = sdf.merge(dfcurrent,
                              'inner',
                              on=None,
                              left_on='PHU_ID',
                              right_on='PHU_NUM')

In [9]:
# Import as a feature collection
# Following import, the layer will need to be added into the web map to appear on the dashboard with the current data. 
# The style may also need updated in order to display the active cases, or whatever other information the user would like.
currentCasesByPHU_fc = gis.content.import_data(currentCasesByPHU,
                                              title='Current COVID-19 Cases by Ontario PHU')
currentCasesByPHU_fc

<Item title:"Current COVID-19 Cases by Ontario PHU" type:Feature Layer Collection owner:razewski.n_CPS>